### 🧙 Lab 7 – Interactive Storytelling App with Choices

In this lab, you'll build a **Gradio app** that generates a branching story — like a role-playing game.  
The AI writes a chapter, then offers **three possible directions**. You choose what happens next.

Each time you click a button, a new chapter is written, and the story continues endlessly.

**🛠️ TODO**

Find the `# TODO` in the code and complete the `ollama.chat(...)` call.  
You'll need to pass a **system message** and a **user prompt** — just like in the earlier labs.

In [ ]:
import gradio as gr
import ollama

MODEL = "gemma3:4b-it-qat"

# Persistent state
context = {
    "story": "",
}

# System prompt to guide generation
SYSTEM_PROMPT = """
You are an interactive story generator.

Each time you're asked to continue, you will:
- Write one new chapter of the story (1–2 paragraphs)
- End with exactly three options for what could happen next

Make the choices diverse and exciting, and make them part of the output. 
Clearly number the options at the end, like:
1. ...
2. ...
3. ...

Do not ask the user to type anything. They will choose by clicking a button.
"""

def start_story(story_idea):
    if story_idea.strip() == "":
        return "Please enter a story idea."
    context["story"] = f"Story idea: {story_idea.strip()}\n\n"
    return continue_story(choice_index=None)

def continue_story(choice_index):
    # Add previous choice info if not the first turn
    if choice_index is not None:
        context["story"] += f"\nUser chose option {choice_index}.\n\n"

    user_prompt = f"""Here is the story so far:

{context['story']}

Please continue the story with one new chapter and three numbered choices.
Each chapter should be no more than 2 paragraphs!
"""

    response = ollama.chat(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt}
        ]
    )

    chapter = response["message"]["content"].strip()
    context["story"] += chapter + "\n"
    return chapter

def launch_app():
    with gr.Blocks() as demo:
        gr.Markdown("## 🧙 Interactive Story Generator with Choices")

        with gr.Row():
            idea_input = gr.Textbox(label="Enter your story idea")
            start_btn = gr.Button("Start Story")

        output_box = gr.Textbox(label="Current Chapter", lines=12)

        with gr.Row():
            btn1 = gr.Button("Choose Option 1")
            btn2 = gr.Button("Choose Option 2")
            btn3 = gr.Button("Choose Option 3")

        start_btn.click(start_story, inputs=idea_input, outputs=output_box)
        btn1.click(lambda: continue_story(1), outputs=output_box)
        btn2.click(lambda: continue_story(2), outputs=output_box)
        btn3.click(lambda: continue_story(3), outputs=output_box)

    demo.launch(server_name="0.0.0.0", server_port=8080)

if __name__ == "__main__":
    launch_app()